In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [2]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="montuno",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

## Add Documents

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader

loader = TextLoader("../data/real-state/proyectos.md")  # Reemplaza con la ruta de tu archivo
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño de cada chunk (en caracteres)
    chunk_overlap=200,  # Superposición entre chunks
    length_function=len,  # Función para calcular la longitud del texto
)
chunks = text_splitter.split_documents(documents)

# 4. Almacenar los embeddings en una base de datos vectorial (FAISS en este caso)
vector_db = vector_store.from_documents(chunks, embeddings)


## Query From vector store

In [31]:
results = vector_store.similarity_search(
    "cash",
    k=2,
    filter={"source": "news"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [32]:
docs = vector_store.similarity_search_with_score("how many cash stolen?",k=2)
print(docs)

[(Document(id='2d28a2ff-b975-4ed0-8a50-927457c642fe', metadata={'tipo': 'sabor'}, page_content='Chocolate'), 1.7875199701499045), (Document(id='292fdbf5-de20-4885-adde-2722b7d5e11e', metadata={'tipo': 'sabor'}, page_content='Chocolate'), 1.7875199701499045)]


## Retriever

In [5]:
retriever = vector_store.as_retriever()

## Prompt template

In [6]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
template = """Responde las siguientes preguntas con base en los siguientes contexto:
Eres un asistente virtual especializado en una agencia inmobiliaria. Tu tarea es proporcionar información precisa y útil sobre los proyectos inmobiliarios disponibles. Utiliza únicamente la información proporcionada en los documentos para responder.

Proyectos disponibles:
1. **Proyecto A**: Residencial "Bosque Verde"
2. **Proyecto B**: Comercial "Plaza Sol"
3. **Proyecto C**: Residencial de lujo "Altos del Mar"

Instrucciones:
- Responde de manera clara y concisa.
- Si no tienes información suficiente, indica que no cuentas con los detalles necesarios.
- Prioriza la información sobre ubicación, precios, características y disponibilidad.

Mas contexto: {context}

Pregunta del usuario: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
response = rag_chain.invoke("tengo presupuesto de 150000 usd, cual proyecto me recomiendas?")
print(response)

Con base en tu presupuesto de 150,000 USD, te recomendaría el Proyecto A: Residencial "Bosque Verde".
